<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 188.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 167.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 256.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 202.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 177.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 204.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 104.5 MB/s eta 0:00:00


In [3]:
%%writefile Lotus.py
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import scrapy
from scrapy import crawler, loader
from itemloaders import processors
import logging
logging.disable("WARNING")

class LotusItem(scrapy.Item):
    category = scrapy.Field(output_processor=processors.TakeFirst())
    name = scrapy.Field(output_processor=processors.TakeFirst())
    price = scrapy.Field(output_processor=processors.TakeFirst())
    info = scrapy.Field()

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--enable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)
        self.collector = loader.ItemLoader(item=LotusItem())

    def start_requests(self):
        urls = [
            "https://www.lotuss.com.my/en/category/meat-poultry/meat/parts-weighted"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_items)

    def parse_items(self, response):
        self.driver.get(response.url)

        elem_present = ""
        while not elem_present:
            try:
                elem_present = self.driver.find_element(By.XPATH,
                    "//div[@class='product-grid-item']"
                )
            except:
                continue

        selector = scrapy.Selector(text=self.driver.page_source)
        category = selector.css("h1#category-title::text").get()
        self.collector.add_value("category", category)

        category_url = selector.css("div.carousel a")
        if len(category_url)!=0:
            yield from response.follow_all(category_url,
                callback=self.parse_items)

        else:
            last_height = self.driver.execute_script(
                "return document.body.scrollHeight"
            )

            while True:
                self.driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight)"
                )
                time.sleep(3)
                new_height = self.driver.execute_script(
                    "return document.body.scrollHeight"
                )
                if new_height==last_height:
                    break
                last_height = new_height

            selector = scrapy.Selector(text=self.driver.page_source)
            items = selector.css("div.product-grid-item")
            for item in items:
                name = item.css("a#product-title::text").get()
                self.collector.add_value("name", name)

                price = item.css("p")[0].css("::text").getall()
                self.collector.add_value("price", "".join(price))

            print(self.collector.load_item())

                # img_url = item.css("img::attr(src)").get()

                # info_page = item.css("a::attr(href)").get()
                # if info_page is not None:
                #     yield response.follow(info_page,
                #         callback=self.parse_info)

                # return l.load_item()

            # info_links = selector.css("div#product-list a")
            # yield from response.follow_all(info_links,
            #     callback=self.parse_info)

    # def parse_info(self, response):
    #     self.driver.get(response.url)

    #     selector = scrapy.Selector(text=self.driver.page_source)
    #     info = selector.css("div.MuiBox-root::text")[2].get()
    #     print(info)
    #     yield info


process = crawler.CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [4]:
%%bash
python Lotus.py

{'category': 'Parts & Weighted',
 'name': "LOTUS'S FRESH AUS LAMB SHOULDER SLICE (5319)",
 'price': 'RM28.97/Kg'}


/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
